In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!pip install tensorflow-gpu==1.15


In [0]:
#input grayscale
#0. Import libraries
#. crop images+ spilit into trainset, valiset, testset arrays
# Import libraries# Impor 
import os,cv2
import random
# import a classifier object
from sklearn.svm import LinearSVC
from numpy import zeros, newaxis
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pylab import rcParams
#import PIL
#print(PIL.PILLOW_VERSION)
import glob
import socket
import csv
import pandas as pd
import keras
from sklearn.utils import shuffle
from keras.utils.np_utils import to_categorical
from keras import utils as np_utils
from keras.models import Sequential

#from tflearn.layers.conv import conv_2d, max_pool_2d
from keras.layers import Dropout

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import load_model
from keras.preprocessing import image

import tensorflow as tf
import keras
from keras.utils import np_utils
from keras import backend as K
from keras import callbacks
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

#ada
from keras.optimizers import SGD,Adadelta

#network
from keras import backend as K

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold

In [0]:
from tensorflow.keras.models import Sequential
from keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.utils.data_utils import Sequence

from keras.layers.normalization import BatchNormalization
import numpy as np
np.random.seed(1000)

In [0]:
#1. Define dataload
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
import keras.backend as K

emotion_list = ["anger", "contempt", "disgust", "fear", "happy", "neutral", "sadness", "surprise"] 
n_classes = len(emotion_list)

class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, list_imgs, batch_size=32, dim=(3, 48, 48), n_channels=3
                        , shuffle=True):
        'Initialization'
        #print(list_imgs)
        #print(type(list_imgs))
        self.dim = dim
        self.batch_size = batch_size
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.list_imgs = list_imgs
        random.shuffle(self.list_imgs)

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_imgs) // self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        img_path = self.list_imgs[index]
        img = cv2.imread(img_path)
        mapping = {}
        y = 9
        #print("len(emotion_list): ",len(emotion_list))
        for x in range(len(emotion_list)):
          if emotion_list[x] in str(img_path):
            y = x
          #  print("y: ",y)
        #print(y)
        #print(num_classes)
        #print(img_path)
        img = cv2.resize(img, (32, 32))
        img = img[ newaxis,:, :, :]
        img = (np.array(img))

        #img = tf.keras.backend.expand_dims(img, axis=0)
        #print("img shape: ",img.shape)
        # y = keras.utils.to_categorical(y, num_classes=self.n_classes)
        #print(y)
        y_label = list()
        y_label.append(y)
        y_label = np.asarray(y_label)
        #print("++++++++++++++++++++++++")
        return (img , y_label)
        
      
    def on_epoch_end(self):
      'Updates indexes after each epoch'
      self.index = np.arange(len(self.list_imgs))
      if self.shuffle == True:
        np.random.shuffle(self.index)
    #Data generators

# Generators
image_folder = "ALL CK+"
original_folder = "/content/gdrive/My Drive/Machine learing/Pre-process/Data Ck plus/ALL CK+"
#main_folder =os.listdir(original_folder)
list_name_img = os.listdir(original_folder)
#list_name_img = next(os.walk(original_folder))[2]
#list_name_img = os.listdir(original_folder)

In [0]:
#2. load list_imgs
nb_img = 0
list_imgs = []
for img_load in list_name_img:
  img_path = original_folder + "/" + img_load
  list_imgs.append(img_path)
  nb_img +=1
  #print("nb_img ",nb_img)

In [0]:
print(len(list_imgs))

In [0]:
#code dot nhien ko chay nen chinh thanh cai ben tren
nb_img = 0
list_imgs = list()
for img in list_name_img:
    if ".png" in img:
        img_path = os.path.join(original_folder, img)
        list_imgs.append(img_path)
        nb_img +=1

#labels = map(lambda x: dict(enumerate(data['target_names']))[x], data['target'])

# randomly shuffle list_imgs
list_imgs = random.shuffle(list_imgs)

In [0]:
print(list_imgs)

In [0]:
#labels = DataGenerator.__getitem__(list_imgs)
labels =[]
print("run training_generator: ")
training_generator = DataGenerator(list_imgs=list_imgs)
print("run validation_generator: ")
validation_generator = DataGenerator(list_imgs=list_imgs)

In [0]:
print(nb_img)

In [0]:
#3. Design model
num_classes = 8
model = Sequential()
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dropout, Dense, Input
from keras.layers import BatchNormalization
from tensorflow.keras import activations

#add kernel_regularizer=l2(0.0005) neu overfitting
model.add(Conv2D(32, (5, 5),strides=(1,1), padding="valid", activation="relu",
                input_shape=[32,32,3]))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(2, (7,7),strides=(1,1),activation ="relu",
                padding="valid"))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256))
model.add(Dropout(0.2))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
model.summary()

In [0]:
#4. Compile
model.compile(loss='binary_crossentropy', optimizer='sgd',metrics=['accuracy'])

In [0]:
model.fit_generator(generator=training_generator,
                    validation_data = validation_generator,  epochs=100)
                    

In [0]:
#5. Run fit+ K-fold
k = 10
len_each_fold = len(list_imgs) // k
for i in range(k-1):
  print("Fold chay thu: ",(i+1))
  list_img_val = list_imgs[i*len_each_fold: (i+1)*len_each_fold]
  list_img_train = list(set(list_imgs) - set(list_img_val))
  print("run training_generator: ")
  training_generator = DataGenerator(list_imgs=list_img_train)
  print("run validation_generator: ")
  validation_generator = DataGenerator(list_imgs=list_img_val)
  model.fit_generator(generator=training_generator,
                    validation_data = validation_generator, epochs=1000)
  print("+++++++++++++++++++++++++++")
  print(" ")
  

In [0]:
#@5. Main code
#Function
#Define k-fold-link: Load link data train, val data
#nb_fold la so vong fold se dung
#def k_fold_link(all_img_link,nb_fold):
nb_fold =10
link_data_val =[]
n = 0
x=[]
mean = 0
val_data = []
train_data = []
#Chia val_train data theo k-fold
k = len()
for n in range(0,nb_fold):
  divide_f = int(nb_img/nb_fold)
  fl_val= divide_f*(n+1)-1
  start_val = divide_f*n
  start_train = divide_f*(n+1)
  print(fl_val)
  x.append(source_img[start_val:fl_val])
  #Gia tri val_data, train_data voi divide_f phan tu
  fl_nb_val = divide_f-1
  fl_nb_train = nb_img-divide_f-1
  val_data[0:fl_nb_val]= (source_img[start_val:fl_val]) 
  train_data[0:fl_nb_train] = source_img[start_train:(nb_img-1)]
  #Chay train_generator
  
  #Chay val_generator
  
  print(" n la ",n)
  print(val_data)
  n=n+1
  #run fit
  #x_train, y_train =train_generator(val_data[0:fl_nb_train],(fl_nb_train+1))
  hist = model.fit(train_generator(val_data[0:fl_nb_train],(fl_nb_train+1)), epochs=10)                 
  #evaluation
  #x_val, y_val =val_generator(val_data[0:fl_nb_val],(fl_nb_val+1))
  score, accuracy =(model.evaluate(val_generator(val_data[0:fl_nb_val],(fl_nb_val+1))))
  print("Acurracy on validation data: ",model.evaluate(val_generator(val_data[0:fl_nb_val],(fl_nb_val+1))) )
  if n==1:
    mean = accuracy
  else:
    mean = (accuracy+ mean)/2
    print(" Mean Accurracy: ", mean)
    
#k_fold_link(source_img, 10)


In [0]:
score, accuracy =(model.evaluate(x_val, y_val))
print(score)
print(accuracy)

In [0]:
#6.save model
model.save('PP_CK+Tr_JAFFE.h5')

In [0]:
import shutil
shutil.copy('/content/PP_CK+Tr_JAFFE.h5', '/content/gdrive/My Drive/Machine learing/Test/Final Pr_3/Result/model') 

In [0]:
reloaded_model = tf.keras.models.load_model('/content/gdrive/My Drive/Machine learing/Test/Final Pr_3/Result/model/PP_CK+Tr_JAFFE.h5')

In [0]:
#7. Evaluation
print("Evaluate on the test data: \n")
results = reloaded_model.evaluate(x_test, y_test)
print("test loss, test accuracy", results)
#print("y test: ", y_test)

In [0]:
#9. Draw matrix result
#9.1.Luu test samples da dung truoc do
np.save('modXtest',x_test)
np.save('modytest',y_test)
#9.2.Load du lieu 
truey=[]
predy=[]
x= np.load('./modXtest.npy')
y= np.load('./modytest.npy')
ypredict = reloaded_model.predict(x)
results_reload = reloaded_model.evaluate(x, y)
print("test loss, test accuracy", results_reload)


In [0]:
import numpy as np
print("x shape: ", x.shape)
print("x 0: ", x[0].shape)
# print("y predict: ", ypredict)
print("y predict shape: ", ypredict.shape)
#  lay label
y_result = np.argmax(ypredict, axis=1) 
print("y result shape:", len(y_result))
print("y result: ", y_result)
y_label = [np.argmax(x) for x in y]
print("y_label: ", y_label)
predy = y_result
true_y = y_label 

np.save('truey', y_label)
np.save('predy', y_result)

In [0]:
#10. Ve matran
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

y_true = np.load('./truey.npy')
y_pred = np.load('./predy.npy')
cm = confusion_matrix(y_true, y_pred)
labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
title='Confusion matrix'
print(cm)

plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title(title)
plt.colorbar()
tick_marks = np.arange(len(labels))
plt.xticks(tick_marks, labels, rotation=45)
plt.yticks(tick_marks, labels)
fmt = 'd'
thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], fmt),
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()